# Averages

The goal of this notebook is to compute summer month averages over the the entire CHIRTS records for each city in our dataset, as well as annual averages. I will need to figure out what the summer months are for each city, which will be the biggest challenge.

## Depdencies

In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import glob
import xarray as xr

## First, let's see if we can load all the data and isolate months

- First needs to open a dir, then open a year, get the months and save it in memory and repeat until we have all the months
- Then we'll need to go back and actually actually isloate the cities by lat long ... this could be a huge pain in the ass

## Dirs and FNS

In [4]:
dir_in = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
# DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/' # ghs ID list
# dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
# fn_out = 'CHIRTS-GHS-Events-Stats'
# time_dim = 'date'
# space_dim = 'ID_HDC_G0'
# Tthresh = 40.6


## Functions

In [249]:
def join_loop(dir_in):
    
    """ Loops through the daily average CHIRTS temp of GHS-UCDB cities annual csv files
    and joins them into one giant x-array data array 
    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
        
    # Get File list
    fn_list = glob.glob(dir_in+'*.csv')
    
    # make one big data frame
    df = pd.DataFrame()
    df['ID_HDC_G0'] = ghs_ids_df['ID_HDC_G0']
    
    # NEED TO ADD WARM SEASON START DATE FAKE IT FOR NOW 
    test_list = [12,1,2,3,4]
    res = [ele for ele in test_list for i in range(int(len(df)/5))]
    df['summer_start'] = res
    

    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        #print(year)
        
        df_temp = pd.read_csv(fn) # read the file in as a df
        #print(df_temp.shape)
    
        df_temp = df_temp.iloc[:,3:] # get only temp columns        
        df = df.join(df_temp)
    
    ### df = df.dropna() # Drop cities w/ no temp record <<<<<------ CHECK AND FIX

    return df
        

In [300]:
def df_to_xr(df):
    "turns data frame from loop into x-array data array"
    
    df.index = df['ID_HDC_G0'] # set index
    df.drop(['ID_HDC_G0'], axis = 1, inplace = True) # drop city ids as a column
    df.drop(['summer_start'], axis = 1, inplace = True) # drop summer starts as a column
    df.columns = pd.to_datetime(df.columns) # cols labels to datetime obj.
    
    temp_np = df.to_numpy() # turn temp cols into an np array
    
    # make x-array
    temp_xr_da = xr.DataArray(temp_np, coords=[df.index, df.columns], dims=['ID_HDC_G0', 'date'])

    return temp_xr_da


In [307]:
def is_amj(month, months):
    "Function to create a boolean to select months from an xr da"
    return (month == months[0]) | (month == months[1]) | (month == months[2])

def summer_select(xr_da, months):
    """Filter x-array data array by select months, args: x-array and list of months as ints"""
    summer = xr_da.sel(date=is_amj(xr_da['date.month'], months)) # subset by warmest three months
    
    return summer

def total_mean(xr_da, months):
    """Finds the mean of the three warmest months over the entire CHIRTS record for each city
    args: xr_da = x-array data array
          months = list of ints"""
    
    summer = temp_xr_da.sel(date=is_amj(temp_xr_da['date.month'], months)) # subset by warmest three months

    mean = summer.mean(dim = 'date') # find mean 
    
    df_out = pd.DataFrame()
    df_out['ID_HDC_G0'] = mean.ID_HDC_G0.values
    df_out['summer_mean'] = mean.values
    
    return df_out


In [301]:
temps = join_loop(dir_in)

In [302]:
temps = temps[temps['summer_start'] == 12]
months = [12,1,2] # This ALWAYS needs to be changed mannually, so summer starts in dec, the months is [12,1,2]

In [303]:
temps.head()

,ID_HDC_G0,summer_start,1983.01.01,1983.01.02,1983.01.03,1983.01.04,1983.01.05,1983.01.06,1983.01.07,1983.01.08,...,2016.12.22,2016.12.23,2016.12.24,2016.12.25,2016.12.26,2016.12.27,2016.12.28,2016.12.29,2016.12.30,2016.12.31
0,5782,12,-43.921947,-33.713450,-33.054974,-25.376320,-23.129280,-21.466286,-19.981667,-15.222391,...,-35.027885,-34.404564,-32.024070,-27.060968,-25.192255,-12.104240,-10.783806,-13.797478,-13.965569,-28.152826
1,3316,12,-4.804248,-3.914425,-7.533999,-12.615092,-7.848448,-6.586810,-5.604166,-5.210299,...,-0.868085,-3.894662,-6.790181,-5.442918,-7.363118,-0.587090,-0.337780,-2.009656,-1.573716,-1.519006
2,5645,12,-23.904118,-17.422953,-13.182008,-14.080582,-15.031302,-17.476360,-21.271954,-12.573436,...,-23.056492,-16.511564,-0.609362,-3.754886,0.640418,1.578676,0.167670,-3.928749,-12.794257,-18.007576
3,3185,12,-0.145468,-0.141592,-4.062948,-2.354636,-1.533712,-0.494162,0.049387,0.149454,...,1.868088,1.287278,0.235062,-0.209609,-0.842849,-1.451431,-0.057235,0.701401,2.318619,2.290848
4,3539,12,-6.093382,-6.081639,-15.774755,-13.649839,-9.780418,-2.830632,1.321785,1.234120,...,0.282529,-0.461020,-1.576864,0.126554,-0.452871,-1.291299,-1.628549,-0.990054,-0.248178,0.390531


In [304]:
# make a xarray
temps_xr = df_to_xr(temps)

In [308]:
temps_mean = total_mean(temps, months)

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/xarray/core/nanops.py:160: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [309]:
temps_mean

,ID_HDC_G0,summer_mean
0,5782,-24.769200
1,3316,-6.876533
2,5645,-18.503008
3,3185,-4.794602
4,3539,-6.782194
...,...,...
13130,13046,19.736792
13131,1116,18.867324
13132,1114,15.127564
13133,1161,15.032221


# NEXT STEP
- Can now do thirty year mean based of the warmest three months
- **NEXT I need to do it annually - likely calc mean every three columns and be sure to stick dec-jan mean onto left year**

# Old Code

In [25]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    

    
    df_id = df[space_dim] # get IDs
    df_temp = df.iloc[:,3:] # get only temp columns
    df_temp.columns = pd.to_datetime(df_temp.columns) # cols labels to datetime obj.
    df_temp.index = df_id # set index values
    df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
    print(len(df_temp_drop))
    
    temp_np = df_temp_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    
    # Note 2019 09 17 changed to xr.Dataset from xr.Dataarray
    temp_xr_da = xr.DataArray(temp_np, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=[space_dim, time_dim])
    
    return temp_xr_da

In [26]:
# Let's open a csv as a file

test_xr = csv_to_xr(dir_in+'GHS-Tmax-DAILY_1983.csv', 'date', 'ID_HDC_G0')

(13135, 368)
13067


In [46]:
test_xr

<xarray.DataArray (ID_HDC_G0: 13067, date: 365)>
array([[-43.921947, -33.71345 , -33.054974, ..., -12.416152, -13.232986,
        -15.403823],
       [ -4.804248,  -3.914425,  -7.533999, ...,  -5.186461, -10.945722,
        -16.29516 ],
       [-23.904118, -17.422953, -13.182008, ..., -12.788978, -11.337886,
        -10.00939 ],
       ...,
       [ 16.028023,  17.73603 ,  20.493294, ...,  14.559421,  15.160739,
         15.184024],
       [ 16.420553,  17.87142 ,  22.519674, ...,  15.680964,  16.169733,
         16.039179],
       [ 16.6943  ,  17.559229,  21.480919, ...,  14.446052,  15.235602,
         14.005591]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) datetime64[ns] 1983-01-01 1983-01-02 ... 1983-12-31

In [49]:
# select months

def is_amj(month, start, end):
    return (month >= start) & (month <= end)

summer = test_xr.sel(date=is_amj(test_xr['date.month'], 6, 8))

mean = summer.mean(dim = 'date')

In [58]:
# Get summer means into a data frame

df_out = pd.DataFrame()
df_out['ID_HDC_G0'] = mean.ID_HDC_G0.values
df_out['summer_mean'] = mean.values

In [59]:
df_out.head()

,ID_HDC_G0,summer_mean
0,5782,14.874282
1,3316,15.306133
2,5645,17.863661
3,3185,17.678628
4,3539,17.044360


In [62]:
# write out test 
DATA_INTERIM = '/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/interim/' # ghs ID list
ghs_ids_fn = 'GHS-UCSB-IDS.csv'
ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
ghs_ids_df

,ID_HDC_G0,CTR_MN_NM
0,5782,Russia
1,3316,Russia
2,5645,Russia
3,3185,Finland
4,3539,Russia
...,...,...
13130,13046,New Zealand
13131,1116,Argentina
13132,1114,Chile
13133,1161,Argentina


In [63]:
# merge them

df_out = df_out.merge(ghs_ids_df, on = 'ID_HDC_G0', how = 'inner')

In [67]:
df_out[df_out['CTR_MN_NM'] == 'Chile']

,ID_HDC_G0,summer_mean,CTR_MN_NM
12559,804,21.743419,Chile
12619,822,19.997928,Chile
12705,876,20.992737,Chile
12770,844,18.920218,Chile
12876,882,18.119088,Chile
12932,884,16.906540,Chile
12939,891,16.776287,Chile
12960,925,12.182786,Chile
12961,908,14.148873,Chile
12962,928,11.873393,Chile


In [ ]:
def stats_loop():
    
    """ Loop through a dir with csvs to apply csv_to_xr and

    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
        
    # Git File list
    fn_list = glob.glob(dir_in+'*.csv')
    
    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print(year)
        
        # read csv as a data array
        temp_xr_da = csv_to_xr(fn, time_dim, space_dim)
        
        # data array to tmax events, out as df
        df_days = tmax_days(temp_xr_da, Tthresh)
        
        # tmax events stats, out as df
        df_out = tmax_stats(df_days)
        
        # merge to get countries
        ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner') 
        
        # write it all out
        ghs_ids_df_out.to_csv(dir_out+fn_out+year+'.csv')

        print(year, 'SAVED!')